# Optimizing trayectories of Omnibot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from numpy import logical_and as npand
from numpy import logical_or as npor
plt.rcParams.update({'font.size': 15})

In [ ]:
from sympy import (symbols, pi, I, E, cos, sin, exp, tan, simplify, expand, factor, collect,
                   apart, cancel, expand_trig, diff, Derivative, Function, integrate, limit,
                   series, Eq, solve, dsolve, Matrix, N, preorder_traversal, Float, solve_linear_system,
                   eye, zeros, lambdify)
from sympy.physics.mechanics import dynamicsymbols, init_vprinting

In [ ]:
from sympy import sqrt

In [ ]:
init_vprinting()

### Using the symbolic formulas to generate the numeric model

In [ ]:
def generic_omnibot_mats(n = 4, null_beta = True, equal_r = True):
    t, r, d, s = symbols('t r d s')
    alpha, beta= dynamicsymbols('alpha beta')
    
    W = Matrix([
        [r, cos(alpha)],
        [0, sin(alpha)]
    ])
    T = Matrix([
        [cos(beta), -sin(beta)],
        [sin(beta), cos(beta)]
    ])
    A = Matrix([
        [1, 0, -d],
        [0, 1, s]
    ])
    WTA= W.inv()@T.inv()@A
    WTA.simplify()
    r_n = WTA[0,:]
    s_n = WTA[1,:]
    
    R_list = []
    S_list = []
    
    for ii in range(n):
        r_ii = r_n.subs(alpha, symbols('alpha_'+str(ii+1)))
        r_ii = r_ii.subs(d, symbols('d_'+str(ii+1)))
        r_ii = r_ii.subs(s, symbols('s_'+str(ii+1)))
        s_ii = s_n.subs(alpha, symbols('alpha_'+str(ii+1)))
        s_ii = s_ii.subs(d, symbols('d_'+str(ii+1)))
        s_ii = s_ii.subs(s, symbols('s_'+str(ii+1)))
        if not equal_r:
            r_ii = r_ii.subs(r, symbols('r_'+str(ii+1)))
            s_ii = s_ii.subs(r, symbols('r_'+str(ii+1)))
        if null_beta:
            r_ii = r_ii.subs(beta, 0)
            s_ii = s_ii.subs(beta, 0)
        else:
            r_ii = r_ii.subs(beta, symbols('beta_'+str(ii+1)))
            s_ii = s_ii.subs(beta, symbols('beta_'+str(ii+1)))
            
        S_list.append(s_ii)
        R_list.append(r_ii)
    
    R = Matrix(R_list)
    S = Matrix(S_list)
    return R, S

In [ ]:
def integerize(expr):
    expr2 = expr
    for a in preorder_traversal(expr):
        if isinstance(a, Float):
            expr2 = expr2.subs(a, round(a))
    return expr2

def roundize(expr,n = 4):
    expr2 = expr
    for a in preorder_traversal(expr):
        if isinstance(a, Float):
            expr2 = expr2.subs(a, round(a,n))
    return expr2

In [ ]:
def dejabot_mats():
    R, S = generic_omnibot_mats()
    L, l = symbols('L l')
    for ii in range(4):
        alpha = pi/4 * (1 - 2 * ((int((ii+1)/2))%2))
        s = L * (1 - 2 * ((int((ii)/2))%2))
        d = l * (1 - 2 * (ii%2))
        R[ii,:] = R[ii,:].subs(symbols('alpha_'+str(ii+1)), alpha)
        R[ii,:] = R[ii,:].subs(symbols('s_'+str(ii+1)), s)
        R[ii,:] = R[ii,:].subs(symbols('d_'+str(ii+1)), d)
        S[ii,:] = S[ii,:].subs(symbols('alpha_'+str(ii+1)), alpha)
        S[ii,:] = S[ii,:].subs(symbols('s_'+str(ii+1)), s)
        S[ii,:] = S[ii,:].subs(symbols('d_'+str(ii+1)), d)
    return integerize(R), integerize(S)

In [ ]:
def dejabot_mats_2():
    R, S = generic_omnibot_mats(null_beta=False)
    L, l = symbols('L_2 l_2')
    s_list = [l, L, -L, l]
    d_list = [L, l, -l, -L]
    for ii in range(4):
        alpha = pi/4 * (1 - 2 * ((int((ii+1)/2))%2))
        s = s_list[ii]
        d = d_list[ii]
        R[ii,:] = R[ii,:].subs(symbols('alpha_'+str(ii+1)), alpha)
        R[ii,:] = R[ii,:].subs(symbols('s_'+str(ii+1)), s)
        R[ii,:] = R[ii,:].subs(symbols('d_'+str(ii+1)), d)
        R[ii,:] = R[ii,:].subs(symbols('beta_'+str(ii+1)), pi/4)
        S[ii,:] = S[ii,:].subs(symbols('alpha_'+str(ii+1)), alpha)
        S[ii,:] = S[ii,:].subs(symbols('s_'+str(ii+1)), s)
        S[ii,:] = S[ii,:].subs(symbols('d_'+str(ii+1)), d)
        S[ii,:] = S[ii,:].subs(symbols('beta_'+str(ii+1)), pi/4)
    return integerize(R), integerize(S)

In [ ]:
t, r, d, s, m, I_z, I_w, l, L, L_2 = symbols('t r d s m I_z I_w l L L_2')
x, y, alpha, beta, sigma, psi, theta= dynamicsymbols('x y alpha beta sigma psi theta')
psi_dot = psi.diff()
x, y, alpha, beta, sigma, psi, psi_dot, theta, t, r, d, s, m, I_z, I_w, l, L, L_2

In [ ]:
q = [x, y, psi] + [dynamicsymbols(f'phi_{i+1}') for i in range(4)]
q_r = Matrix([x, y, psi])
q = Matrix(q)
Gamma = [dynamicsymbols(f'tau_{i+1}') for i in range(4)]
Gamma = Matrix(Gamma)

q, Gamma

In [ ]:
M_w = eye(4)*I_w
M_r = Matrix([
    [m, 0, 0],
    [0, m, 0],
    [0, 0, I_z]
])
R_psi = Matrix([
    [cos(psi), -sin(psi), 0],
    [sin(psi), cos(psi), 0],
    [0, 0, 1]
])
R, S = dejabot_mats_2()

M_r, M_w, R_psi, R

In [ ]:
H = M_r + R_psi@R.T@M_w@R@R_psi.T
H = simplify(H)
K = R_psi@R.T@M_w@R@R_psi.diff().T
K = simplify(K)
F_a = R_psi@R.T@Gamma
F_a_0 = F_a[0].factor(sin(psi), cos(psi))
F_a_1 = F_a[1].factor(sin(psi), cos(psi))
F_a_2 = F_a[2].factor()
F_a = Matrix([F_a_0, F_a_1, F_a_2])
#A = R_psi@R.T COMPROBAR
H_inv = simplify(H.inv())
R_inv = simplify(R.pinv())
H, K, F_a, R_inv#A

In [ ]:
get_h_raw = lambdify([m, I_w, I_z, L_2, r], H)
get_k_raw = lambdify([I_w, r, psi_dot], K)
#get_a_raw = lambdify([psi, L_2, r], A)
get_h_inv_raw = lambdify([m, I_w, I_z, L_2, r], H_inv)

## Energy and Direction Study

![Esquema de Omnibot](images_dejabot/Omnibot_con_gamma.png)

#### Parameter encoding in functions:
$$Geom = [l, L, r]$$
$$Mass = [m, I_w,I_z]$$
$$q = [x, y, \psi]$$ where q is $q_r$

In [ ]:
geom = [0.2096, 0.2096, 0.0667]
mass = [15.75, 0.00266, 0.461]
q_0 = np.array([0, 0, 0])

In [ ]:
def get_h(geom, mass):
    l, L, r = geom
    m, I_w, I_z = mass
    L_2 = (L+l)/(2**0.5)
    return get_h_raw(m, I_w, I_z, L_2, r)

def get_k(geom, mass, q_dot):
    l, L, r = geom
    m, I_w, I_z = mass
    x_dot, y_dot, psi_dot = q_dot
    return get_k_raw(I_w, r, psi_dot)

def get_a(geom, q):
    l, L, r = geom
    x, y, psi = q
    L_2 = (L+l)/(2**0.5)
    return get_a_raw(psi, L_2, r)

def get_h_inv(geom, mass):
    l, L, r = geom
    m, I_w, I_z = mass
    L_2 = (L+l)/(2**0.5)
    return get_h_inv_raw(m, I_w, I_z, L_2, r)


In [ ]:
# L_2:
2**0.5 * 0.2096

In [ ]:
#A_arr = get_a(geom, q_0)
#A_arr

In [ ]:
simplify(R*r/(sqrt(2)))

#### Lets work with in coordinate system 2

In [ ]:
w, w_1, w_2, a, a_1, a_2 = symbols('w w_1 w_2 a a_1 a_2')
q_d = Matrix([w_1, w_2, psi.diff()])
q_d_d = Matrix([a_1, a_2, psi.diff(t,2)])
q_d_d, q_d

In [ ]:
q_d_w = simplify(R @ q_d)
q_d_w

In [ ]:
simplify(q_d_w*r/(sqrt(2)))

In [ ]:
def max_speed_axes_2(psi_dot):
    L_2 = 0.29642
    r = 0.0667
    phi_dot_max = 2 * np.pi * 7000/(49*60)
    return(phi_dot_max * r/(2**0.5) - np.abs(L_2 * psi_dot))

In [ ]:
phi_dot_max = 2 * np.pi * 7000/(49*60)
psi_dot_max = phi_dot_max * 0.0667/(2**0.5 * 0.29642)
psi_dot_max

In [ ]:
from matplotlib import cm

In [ ]:
percent = [0, 0.25, 0.5, 0.75, 0.95]
viridis = cm.get_cmap('viridis', 12)

fig, ax = plt.subplots(figsize = [8,8])
ax.set_title('Speed Envelope', picker=True)

ax.set_xlim([-0.8, 0.8])
ax.set_ylim([-0.8, 0.8])
ax.set_xlabel('$w_1$, m/s')
ax.set_ylabel('$w_2$, m/s')

for ii in percent:
    psi_val = ii * psi_dot_max
    mx_spd = max_speed_axes_2(psi_val)

    x_coord = [mx_spd, mx_spd, -mx_spd, -mx_spd, mx_spd]
    y_coord = [mx_spd, -mx_spd, -mx_spd, mx_spd, mx_spd]
    plt.plot(x_coord, y_coord, color = viridis(ii), label = str(ii)+'$\dot{\psi}_{max}$')
plt.grid()
plt.legend()

## Electric considerations

Complete Electric Motor Model

$$ V = K_m N\dot{\phi} + Ri$$
$$ \tau_m = N K_ei\mu_{trans} - \tau_r$$
$$ \tau_r = a \dot{\phi} + b·sign(\dot{\phi}) $$

Simplified Electric Motor Model

$$ V = K_m N\dot{\phi} + Ri$$
$$ \tau_m = N K_ei$$

In [ ]:
a, b, k_e, k_m, n, r_e, tau_m = symbols('a b K_e K_m N R_e tau_m')

a, b, k_e, k_m, n, r_e,tau_m

In [ ]:
from sympy.functions import sign

Movimiento en uniforme en ejes 2: : $$\vec{w} = \left[\begin{matrix}w_1\\w_2\\0\end{matrix}\right]$$

In [ ]:
q_d, q_d_w

In [ ]:
phi = dynamicsymbols('\phi')
mu = symbols('\mu')
phi_dot = diff(phi)
phi, phi_dot, mu

In [ ]:
i = tau_m/(k_e * n) # * mu
i

In [ ]:
v = k_m * n * phi_dot + r_e * i
v

In [ ]:
pow_e = expand(i*v)
pow_e

In [ ]:
pow_e_simp_num_nofric = pow_e.subs(k_e, 0.0337).subs(k_m, 0.03148).subs(r_e, 1.84615).subs(n, 49)
roundize(pow_e_simp_num_nofric,3)

In [ ]:
pow_e_simp_f = lambdify([phi_dot, tau_m], pow_e_simp_num_nofric)

### Max and min torque as $f(\dot\phi)$ with simplified model

In [ ]:
v_max = symbols('V_{max}')
v_max

In [ ]:
i_v_max = (v_max - k_m * n *  phi_dot)/r_e
i_v_max

In [ ]:
tau_m_v_max = expand(n * k_e * i_v_max)
tau_m_v_max

In [ ]:
tau_m_v_max_num_simp = tau_m_v_max.subs(k_e, 0.0337).subs(k_m, 0.03148).subs(r_e, 1.84615).subs(n, 49).subs(v_max, 24)
roundize(tau_m_v_max_num_simp,3)

In [ ]:
tau_m_v_min_num_simp = tau_m_v_max.subs(k_e, 0.0337).subs(k_m, 0.03148).subs(r_e, 1.84615).subs(n, 49).subs(v_max, -24)
roundize(tau_m_v_min_num_simp,3)

In [ ]:
tau_m_v_zero_num_simp = tau_m_v_max.subs(k_e, 0.0337).subs(k_m, 0.03148).subs(r_e, 1.84615).subs(n, 49).subs(v_max, 0)
roundize(tau_m_v_zero_num_simp,3)

In [ ]:
tau_m_v_max_f = lambdify([phi_dot], tau_m_v_max_num_simp)
tau_m_v_min_f = lambdify([phi_dot], tau_m_v_min_num_simp)
tau_m_v_zero_f = lambdify([phi_dot], tau_m_v_zero_num_simp)

In [ ]:
phi_dot_max_no_fricc = 15.558955475455715
tau_m_v_max_f(phi_dot_max_no_fricc), tau_m_v_min_f(-phi_dot_max_no_fricc)

In [ ]:
NN = 100
phi_dot_arr_simp = np.linspace(0, phi_dot_max_no_fricc, NN)
_1 = np.ones([NN,1])
phi_dot_mat_simp = _1 @ np.expand_dims(phi_dot_arr_simp,0)
tau_m_mat_simp = np.zeros_like(phi_dot_mat_simp)
for ii in range(NN):
    tau_m_mat_simp[:,ii] = np.linspace(tau_m_v_min_f(phi_dot_arr_simp[ii]), tau_m_v_max_f(phi_dot_arr_simp[ii]), NN)
pow_e_mat_simp = pow_e_simp_f(phi_dot_mat_simp, tau_m_mat_simp)

In [ ]:
fig, ax = plt.subplots(figsize = [8,12])
ax.set_title('Potencia eléctrica del motor sin rozamiento', picker=True)
ax.contourf(phi_dot_mat_simp, tau_m_mat_simp, pow_e_mat_simp, levels = 30)
CS = ax.contour(phi_dot_mat_simp, tau_m_mat_simp, pow_e_mat_simp, colors = 'k',
#               colors = ['w','w','w','w','w','k','k','k','k','k'], 
               levels = [-50,-20,-10,0,10,20,50,100,150,200,300,400,500]
#                levels = 20
               )
ax.clabel(CS, inline=1, fontsize=10, fmt = '%i W')
plt.plot(phi_dot_mat_simp[-1,:],tau_m_mat_simp[-1,:], 'k')
plt.plot(phi_dot_mat_simp[0,:],tau_m_mat_simp[0,:], 'k')

ax.set_xlabel('$\dot{\phi}$')
ax.set_ylabel('$\\tau_m$')

### Comparación con modelo completo

In [ ]:
i = (tau_m + a * phi_dot + b * sign(phi_dot))/(k_e * mu * n)
i

In [ ]:
v = k_m * n * phi_dot + r_e * i
v

In [ ]:
pow_e = expand(i*v)
pow_e

In [ ]:
#pow_e = pow_e.replace(sign(phi_dot)**2, 1)
#pow_e_simp = collect(pow_e.replace(sign(phi_dot), 1), phi_dot)
pow_e_simp = collect(pow_e, phi_dot)
pow_e_simp

In [ ]:
pow_e_simp_num = pow_e_simp.subs(a, 0.01).subs(b, 0.4).subs(k_e, 0.0337).subs(k_m, 0.03148).subs(r_e, 1.84615).subs(n, 49).subs(mu, 0.6)
roundize(pow_e_simp_num,3)

In [ ]:
pow_e_f = lambdify([phi_dot, tau_m], pow_e_simp_num)

### Max torque as $f(\dot\phi)$ with complete model

In [ ]:
i_v_max = (v_max - k_m * n *  phi_dot)/r_e
i_v_max

In [ ]:
tau_r = a * phi_dot + b * sign(phi_dot)
tau_r

In [ ]:
tau_m_v_max = expand(n * k_e * i_v_max * mu - tau_r)
tau_m_v_max

In [ ]:
tau_m_v_max_num = tau_m_v_max.subs(k_e, 0.0337).subs(k_m, 0.03148).subs(r_e, 1.84615).subs(n, 49).subs(a, 0.01).subs(b, 0.4).subs(mu, 0.6).subs(v_max, 24)
roundize(tau_m_v_max_num,3)

In [ ]:
tau_m_v_min_num = tau_m_v_max.subs(k_e, 0.0337).subs(k_m, 0.03148).subs(r_e, 1.84615).subs(n, 49).subs(v_max, -24).subs(a, 0.01).subs(b, 0.4).subs(mu, 0.6)
roundize(tau_m_v_min_num,3)

In [ ]:
tau_m_v_zero_num = tau_m_v_max.subs(k_e, 0.0337).subs(k_m, 0.03148).subs(r_e, 1.84615).subs(n, 49).subs(v_max, 0).subs(a, 0.01).subs(b, 0.4).subs(mu, 0.6)
roundize(tau_m_v_zero_num,3)

In [ ]:
tau_m_v_max_f_comp = lambdify([phi_dot], tau_m_v_max_num)
tau_m_v_min_f_comp = lambdify([phi_dot], tau_m_v_min_num)
tau_m_v_zero_f_comp = lambdify([phi_dot], tau_m_v_zero_num)

In [ ]:
phi_dot_max = 14.895826088293878
tau_m_v_max_f_comp(phi_dot_max),tau_m_v_min_f_comp(-phi_dot_max)

In [ ]:
NN = 100
phi_dot_arr = np.linspace(0, phi_dot_max, NN)
_1 = np.ones([NN,1])
phi_dot_mat = _1 @ np.expand_dims(phi_dot_arr,0)
tau_m_mat = np.zeros_like(phi_dot_mat)
for ii in range(NN):
    tau_m_mat[:,ii] = np.linspace(tau_m_v_min_f_comp(phi_dot_arr[ii]), tau_m_v_max_f_comp(phi_dot_arr[ii]), NN)
pow_e_mat = pow_e_f(phi_dot_mat, tau_m_mat)

In [ ]:
fig, ax = plt.subplots(figsize = [8,12])
ax.set_title('Potencia eléctrica del motor con rozamiento', picker=True)
ax.contourf(phi_dot_mat, tau_m_mat, pow_e_mat, levels = 30)
CS = ax.contour(phi_dot_mat, tau_m_mat, pow_e_mat,  colors = 'k',
#               colors = ['w','w','w','w','w','k','k','k','k','k'], 
               levels = [-50,-20,-10,0,10,20,50,100,150,200,300,400,500]
#                levels = 20
               )
ax.clabel(CS, inline=1, fontsize=10, fmt = '%i W')
plt.plot(phi_dot_mat[0,:],tau_m_mat[0,:], 'k')
plt.plot(phi_dot_mat[-1,:],tau_m_mat[-1,:], 'k')

ax.set_xlabel('$\dot{\phi}$')
ax.set_ylabel('$\\tau_m$')

In [ ]:
plt.figure(figsize = [15,12])

ax = plt.subplot(1,2,1)
ax.set_title('Motor electric power, without friction', picker=True)
#ax.contourf(phi_dot_mat_simp, tau_m_mat_simp, pow_e_mat_simp, levels = 30)
plt.plot(phi_dot_mat_simp[0,:],tau_m_mat_simp[0,:], 'k')
plt.plot(phi_dot_mat_simp[-1,:],tau_m_mat_simp[-1,:], 'k')
CS = ax.contour(phi_dot_mat_simp, tau_m_mat_simp, pow_e_mat_simp, 
#               colors = ['w','w','w','w','w','k','k','k','k','k'], 
               levels = [-50,-20,0,20,50,100,150,200,300,400,500])
ax.clabel(CS, inline=1, fontsize=10, fmt = '%i W')

ax.set_xlabel('$\dot{\phi}, rad/s$')
ax.set_ylabel('$\\tau_m, Nm$')
ax.set_xlim([0, 16])
ax.set_ylim([-45, 22.5])
plt.grid()

ax = plt.subplot(1,2,2)
ax.set_title('Motor electric power, with friction', picker=True)
#ax.contourf(phi_dot_mat, tau_m_mat, pow_e_mat, levels = 30)
CS = ax.contour(phi_dot_mat, tau_m_mat, pow_e_mat, 
#               colors = ['w','w','w','w','w','k','k','k','k','k'], 
               levels = [-50,-20,0,20,50,100,150,200,300,400,500])
ax.clabel(CS, inline=1, fontsize=10, fmt = '%i W')
plt.plot(phi_dot_mat[0,:],tau_m_mat[0,:], 'k')
plt.plot(phi_dot_mat[-1,:],tau_m_mat[-1,:], 'k')

ax.set_xlabel('$\dot{\phi}, rad/s$')
ax.set_ylabel('$\\tau_m, Nm$')
ax.set_xlim([0, 16])
ax.set_ylim([-45, 22.5])
plt.grid()


## Situación y ecuación principal

$$ Ha+Kw=R^T\Gamma$$
Si $\dot\psi = 0$:
$$ Ha = R^T\Gamma$$
$$ \Gamma = R^{-1T}Ha$$

In [ ]:
q_d_d

In [ ]:
R_inv.T

In [ ]:
H

In [ ]:
gamma = R_inv.T @ H @ q_d_d.subs( psi.diff(t,2),0)
gamma

## Potencia eléctrica en función de la velocidad de las ruedas

In [ ]:
q_d_w.subs( psi.diff(t,1),0)

In [ ]:
roundize(pow_e_simp_num_nofric,3)

In [ ]:
total_p = 0
for iii in range(4):
    total_p += pow_e_simp_num_nofric.subs(phi_dot, q_d_w[iii]).subs( psi.diff(t,1),0).subs(tau_m, gamma[iii])
total_p = simplify(total_p.subs(sqrt(2),2**0.5).subs(r, 0.0667).subs(L_2, 0.29642).subs(m,15.75).subs(I_w, 0.00266))
roundize(total_p, 2)

In [ ]:
total_pow_e_f_simp = lambdify([w_1, w_2, a_1, a_2], total_p)

### Comparación con el modelo con rozamiento

In [ ]:
roundize(pow_e_simp_num,3)

In [ ]:
total_p_confric = 0
for iii in range(4):
    total_p_confric += pow_e_simp_num.subs(phi_dot, q_d_w[iii]).subs(psi.diff(t,1),0).subs(tau_m, gamma[iii])
total_p_confric = simplify(total_p_confric.subs(sqrt(2),2**0.5).subs(r, 0.0667).subs(L_2, 0.29642).subs(m,15.75).subs(I_w, 0.00266))
roundize(total_p_confric, 2)

In [ ]:
total_pow_e_f = lambdify([w_1, w_2, a_1, a_2], total_p_confric)

### Model modified:


$$ V = K_m N\dot{\phi} + Ri$$
$$ \tau_m = N K_ei\mu_{trans} - \tau_r$$
$$ \tau_r = a \dot{\phi} + b·sign(\dot{\phi}) $$
$$ P_{e}=\begin{cases}
               Vi \text{  if  } Vi\geq 0\\
               0 \text{  if  } Vi < 0
            \end{cases}$$

In [ ]:
def only_positive(fun):
    def wrap(*args, **kwargs):
        result = fun(*args, **kwargs)
        result_fixed = np.where(result >=0, result, 0)
        return result_fixed
    return wrap

In [ ]:
def ejemplo(x):
    return x

ejemplo_2 = only_positive(ejemplo)
for i in range(10):
    print(ejemplo_2(i-5))

In [ ]:
gamma

In [ ]:
pow_e_simp_num

In [ ]:
def total_pow_e_f_mod(w1, w2, a1, a2):
    p = 0
    for iii in range(4):
        p_motor = pow_e_simp_num.subs(phi_dot, q_d_w[iii]).subs(psi.diff(t,1),0).subs(tau_m, gamma[iii])
        p_motor = simplify(p_motor.subs(sqrt(2),2**0.5).subs(r, 0.0667).subs(L_2, 0.29642).subs(m,15.75).subs(I_w, 0.00266))
        p_motor_f = only_positive(lambdify([w_1, w_2, a_1, a_2], p_motor))
        p += p_motor_f(w1, w2, a1, a2)
    return p

### Max and min acceleration

$$Ha = R^T\Gamma$$
$$a = H^{-1}R^T\Gamma$$

In [ ]:
H_inv

In [ ]:
q_d_w.subs( psi.diff(t,1),0)

In [ ]:
tau_m_v_max_num_simp

In [ ]:
max_gamma_simp = Matrix([0,0,0,0])
min_gamma_simp = Matrix([0,0,0,0])
zero_gamma_simp = Matrix([0,0,0,0])
for iii in range(4):
    max_gamma_simp[iii] = tau_m_v_max_num_simp.subs(phi_dot, q_d_w.subs( psi.diff(t,1),0)[iii]).subs(sqrt(2),2**0.5).subs(r, 0.0667)
    min_gamma_simp[iii] = tau_m_v_min_num_simp.subs(phi_dot, q_d_w.subs( psi.diff(t,1),0)[iii]).subs(sqrt(2),2**0.5).subs(r, 0.0667)
    zero_gamma_simp[iii] = tau_m_v_zero_num_simp.subs(phi_dot, q_d_w.subs( psi.diff(t,1),0)[iii]).subs(sqrt(2),2**0.5).subs(r, 0.0667)
roundize(max_gamma_simp,3),roundize(min_gamma_simp,3),roundize(zero_gamma_simp,3)

In [ ]:
max_a_simp = H_inv@R.T@max_gamma_simp
min_a_simp = H_inv@R.T@min_gamma_simp
zero_a_simp = H_inv@R.T@zero_gamma_simp
max_a_simp = max_a_simp.subs(sqrt(2),2**0.5).subs(r, 0.0667).subs(m,15.75).subs(I_w, 0.00266)
min_a_simp = min_a_simp.subs(sqrt(2),2**0.5).subs(r, 0.0667).subs(m,15.75).subs(I_w, 0.00266)
zero_a_simp = zero_a_simp.subs(sqrt(2),2**0.5).subs(r, 0.0667).subs(m,15.75).subs(I_w, 0.00266)
roundize(max_a_simp,3),roundize(min_a_simp,3),roundize(zero_a_simp,3)

In [ ]:
max_a_simp_f = lambdify([w_1],max_a_simp[0])
min_a_simp_f = lambdify([w_1],min_a_simp[0])
zero_a_simp_f = lambdify([w_1],zero_a_simp[0])

In [ ]:
max_speed_ax_simp = phi_dot_max_no_fricc*0.0667/(2**0.5)
max_speed_ax_simp, max_a_simp[0].subs(w_1, max_speed_ax_simp)

In [ ]:
NN = 100
speed_arr_simp = np.linspace(0, max_speed_ax_simp, NN)
_1 = np.ones([NN,1])
speed_mat_simp = _1 @ np.expand_dims(speed_arr_simp,0)
acc_mat_simp = np.zeros_like(speed_mat_simp)
for ii in range(NN):
    acc_mat_simp[:,ii] = np.linspace(min_a_simp_f(speed_arr_simp[ii]), max_a_simp_f(speed_arr_simp[ii]), NN)
total_pow_e_mat_simp = total_pow_e_f_simp(speed_mat_simp,0,acc_mat_simp,0)

In [ ]:
fig, ax = plt.subplots(figsize = [8,12])
ax.set_title('Potencia eléctrica total sin rozamiento', picker=True)
ax.contourf(speed_mat_simp, acc_mat_simp, total_pow_e_mat_simp, levels = 30)
CS = ax.contour(speed_mat_simp, acc_mat_simp, total_pow_e_mat_simp, colors = 'k',
#               colors = ['w','w','w','w','w','k','k','k','k','k'], 
               levels = [-100,-50,-20,0,20,50,100,200,300,400,600,800,1000]
#                levels = 20
               )
ax.clabel(CS, inline=1, fontsize=10, fmt = '%i W')
#plt.plot(phi_dot_mat_simp[-1,:],tau_m_mat_simp[-1,:], 'k')
#plt.plot(phi_dot_mat_simp[0,:],tau_m_mat_simp[0,:], 'k')

ax.set_xlabel('$w_1, m/s$')
ax.set_ylabel('$a_1, m/s^2$')

### Comparación con modelo completo

In [ ]:
tau_m_v_max_num

In [ ]:
max_gamma = Matrix([0,0,0,0])
min_gamma = Matrix([0,0,0,0])
zero_gamma = Matrix([0,0,0,0])
for iii in range(4):
    max_gamma[iii] = tau_m_v_max_num.subs(phi_dot, q_d_w.subs( psi.diff(t,1),0)[iii]).subs(sqrt(2),2**0.5).subs(r, 0.0667)
    min_gamma[iii] = tau_m_v_min_num.subs(phi_dot, q_d_w.subs( psi.diff(t,1),0)[iii]).subs(sqrt(2),2**0.5).subs(r, 0.0667)
    zero_gamma[iii] = tau_m_v_zero_num.subs(phi_dot, q_d_w.subs( psi.diff(t,1),0)[iii]).subs(sqrt(2),2**0.5).subs(r, 0.0667)
roundize(max_gamma,3),roundize(min_gamma,3),roundize(zero_gamma,3)

In [ ]:
max_a = H_inv@R.T@max_gamma
min_a = H_inv@R.T@min_gamma
zero_a = H_inv@R.T@zero_gamma
max_a = max_a.subs(sqrt(2),2**0.5).subs(r, 0.0667).subs(m,15.75).subs(I_w, 0.00266)
min_a = min_a.subs(sqrt(2),2**0.5).subs(r, 0.0667).subs(m,15.75).subs(I_w, 0.00266)
zero_a = zero_a.subs(sqrt(2),2**0.5).subs(r, 0.0667).subs(m,15.75).subs(I_w, 0.00266)
roundize(max_a,3),roundize(min_a,3),roundize(zero_a,3)

In [ ]:
max_a_f = lambdify([w_1],max_a[0])
min_a_f = lambdify([w_1],min_a[0])
zero_a_f = lambdify([w_1],zero_a[0])

In [ ]:
max_speed_ax = phi_dot_max*0.0667/(2**0.5)
max_speed_ax, max_a[0].subs(w_1, max_speed_ax)

In [ ]:
phi_dot_max

In [ ]:
NN = 300
speed_arr = np.linspace(0, max_speed_ax, NN)
_1 = np.ones([NN,1])
speed_mat = _1 @ np.expand_dims(speed_arr,0)
acc_mat = np.zeros_like(speed_mat)
for ii in range(NN):
    acc_mat[:,ii] = np.linspace(min_a_f(speed_arr[ii]), max_a_f(speed_arr[ii]), NN)
total_pow_e_mat = total_pow_e_f(speed_mat,0,acc_mat,0)
total_pow_e_mat_mod = total_pow_e_f_mod(speed_mat,0,acc_mat,0)

In [ ]:
plt.figure(figsize = [18,12])

ax = plt.subplot(1,3,1)
ax.set_title('Total electric power, without friction', picker=True)
#ax.contourf(phi_dot_mat_simp, tau_m_mat_simp, pow_e_mat_simp, levels = 30)
plt.plot(speed_mat_simp[0,:],acc_mat_simp[0,:], 'k')
plt.plot(speed_mat_simp[-1,:],acc_mat_simp[-1,:], 'k')
CS = ax.contour(speed_mat_simp, acc_mat_simp, total_pow_e_mat_simp, 
#               colors = ['w','w','w','w','w','k','k','k','k','k'], 
               levels = [-100,-50,-20,0,20,50,100,200,300,400,600,800,1000])
ax.clabel(CS, inline=1, fontsize=10, fmt = '%i W')

ax.set_xlabel('$w_1, m/s$')
ax.set_ylabel('$a_1, m/s^2$')
ax.set_xlim([0, 0.75])
ax.set_ylim([-105, 55])
plt.grid()

ax = plt.subplot(1,3,2)
ax.set_title('Total electric power, with friction', picker=True)
#ax.contourf(phi_dot_mat_simp, tau_m_mat_simp, pow_e_mat_simp, levels = 30)
plt.plot(speed_mat[0,:],acc_mat[0,:], 'k')
plt.plot(speed_mat[-1,:],acc_mat[-1,:], 'k')
CS = ax.contour(speed_mat, acc_mat, total_pow_e_mat, 
#               colors = ['w','w','w','w','w','k','k','k','k','k'], 
               levels = [-100,-50,0,50,100,200,300,400,600,800,1000])
ax.clabel(CS, inline=1, fontsize=10, fmt = '%i W')

ax.set_xlabel('$w_1, m/s$')
#ax.set_ylabel('$a_1, m/s^2$')
ax.set_xlim([0, 0.75])
ax.set_ylim([-105, 55])
plt.grid()


ax = plt.subplot(1,3,3)
ax.set_title('Total electric power, mod model', picker=True)
#ax.contourf(phi_dot_mat_simp, tau_m_mat_simp, pow_e_mat_simp, levels = 30)
plt.plot(speed_mat[0,:],acc_mat[0,:], 'k')
plt.plot(speed_mat[-1,:],acc_mat[-1,:], 'k')
CS = ax.contour(speed_mat, acc_mat, total_pow_e_mat_mod, 
#               colors = ['w','w','w','w','w','k','k','k','k','k'], 
               levels = [-100,-50,0,50,100,200,300,400,600,800,1000])
ax.clabel(CS, inline=1, fontsize=10, fmt = '%i W')

ax.set_xlabel('$w_1, m/s$')
#ax.set_ylabel('$a_1, m/s^2$')
ax.set_xlim([0, 0.75])
ax.set_ylim([-105, 55])
plt.grid()


## Acceleration from static

$$\vec{w} = \vec{0}$$

In [ ]:
roundize(total_p_confric.subs(w_1,0).subs(w_2,0), 2)

In [ ]:
roundize(total_p.subs(w_1,0).subs(w_2,0), 2)

In [ ]:
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle, Circle

In [ ]:
def cuadrado(x_cent, y_cent, lado, fill=False, linestyle = ':'):
    return Rectangle((x_cent-lado/2, y_cent - lado/2), lado, lado, fill=fill, linestyle = linestyle)

In [ ]:
NN = 100
max_a_val_simp = max_a_simp_f(0)
acc_array_simp = np.linspace(-max_a_val_simp, max_a_val_simp, NN)
acc_XX_simp, acc_YY_simp = np.meshgrid(acc_array_simp, acc_array_simp)
total_pow_e_mat_simp = total_pow_e_f_simp(0,0,acc_XX_simp,acc_YY_simp)

max_a_val = max_a_f(0)
acc_array = np.linspace(-max_a_val, max_a_val, NN)
acc_XX, acc_YY = np.meshgrid(acc_array, acc_array)
total_pow_e_mat = total_pow_e_f(0,0,acc_XX,acc_YY)
total_pow_e_mat_mod = total_pow_e_f_mod(0,0,acc_XX,acc_YY)

plt.figure(figsize = [22,7])
xlims = [-52, 52]
ylims = [-52, 52]

ax = plt.subplot(1,3,1)
ax.set_title('Total electric power, without friction', picker=True)
#ax.contourf(phi_dot_mat_simp, tau_m_mat_simp, pow_e_mat_simp, levels = 30)
#plt.plot(speed_mat_simp[0,:],acc_mat_simp[0,:], 'k')
#plt.plot(speed_mat_simp[-1,:],acc_mat_simp[-1,:], 'k')
CS = ax.contour(acc_XX_simp,acc_YY_simp, total_pow_e_mat_simp, 
#               colors = ['w','w','w','w','w','k','k','k','k','k'], 
               levels = 12) #[0,10, 20,50,100,200,300,400])
ax.clabel(CS, inline=1, fontsize=10, fmt = '%i W')

ax.set_xlabel('$a_1, m/s^2$')
ax.set_ylabel('$a_2, m/s^2$')
ax.set_xlim(xlims)
ax.set_ylim(ylims)
rect = cuadrado(0,0, 2* max_a_val_simp)
ax.add_patch(rect)
plt.grid()


ax = plt.subplot(1,3,2)
ax.set_title('Total electric power, with friction', picker=True)
#ax.contourf(phi_dot_mat_simp, tau_m_mat_simp, pow_e_mat_simp, levels = 30)
#plt.plot(speed_mat_simp[0,:],acc_mat_simp[0,:], 'k')
#plt.plot(speed_mat_simp[-1,:],acc_mat_simp[-1,:], 'k')
CS = ax.contour(acc_XX,acc_YY, total_pow_e_mat, 
#               colors = ['w','w','w','w','w','k','k','k','k','k'], 
               levels = 12) #[0,10, 20,50,100,200,300,400])
ax.clabel(CS, inline=1, fontsize=10, fmt = '%i W')

ax.set_xlabel('$a_1, m/s^2$')
ax.set_ylabel('$a_2, m/s^2$')
ax.set_xlim(xlims)
ax.set_ylim(ylims)
rect = cuadrado(0,0, 2* max_a_val)
ax.add_patch(rect)
plt.grid()


ax = plt.subplot(1,3,3)
ax.set_title('Total electric power, modified model', picker=True)
#ax.contourf(phi_dot_mat_simp, tau_m_mat_simp, pow_e_mat_simp, levels = 30)
#plt.plot(speed_mat_simp[0,:],acc_mat_simp[0,:], 'k')
#plt.plot(speed_mat_simp[-1,:],acc_mat_simp[-1,:], 'k')
CS = ax.contour(acc_XX,acc_YY, total_pow_e_mat_mod, 
#               colors = ['w','w','w','w','w','k','k','k','k','k'], 
               levels = 12) #[0,10, 20,50,100,200,300,400])
ax.clabel(CS, inline=1, fontsize=10, fmt = '%i W')

ax.set_xlabel('$a_1, m/s^2$')
ax.set_ylabel('$a_2, m/s^2$')
ax.set_xlim(xlims)
ax.set_ylim(ylims)
rect = cuadrado(0,0, 2* max_a_val)
ax.add_patch(rect)
plt.grid()

### Moving at half speed in $w_1$

In [ ]:
NN = 101
speed_simp = max_speed_ax_simp/2
max_a_val_simp = max_a_simp_f(speed_simp)
min_a_val_simp = min_a_simp_f(speed_simp)
max_a_lat_val_simp = max_a_simp_f(0)
acc_array_x_simp = np.linspace(min_a_val_simp, max_a_val_simp, NN)
acc_array_y_simp = np.linspace(-max_a_lat_val_simp, max_a_lat_val_simp, NN)
acc_XX_simp, acc_YY_simp = np.meshgrid(acc_array_x_simp, acc_array_y_simp)
total_pow_e_mat_simp = total_pow_e_f_simp(speed_simp,0,acc_XX_simp,acc_YY_simp)

plt.figure(figsize = [22,6.5])
xlims = [-78, 28]
ylims = [-52, 52]

ax = plt.subplot(1,3,1)
ax.set_title('Total electric power, without friction', picker=True)
#ax.contourf(phi_dot_mat_simp, tau_m_mat_simp, pow_e_mat_simp, levels = 30)
#plt.plot(speed_mat_simp[0,:],acc_mat_simp[0,:], 'k')
#plt.plot(speed_mat_simp[-1,:],acc_mat_simp[-1,:], 'k')
CS = ax.contour(acc_XX_simp,acc_YY_simp, total_pow_e_mat_simp, 
#               colors = ['w','w','w','w','w','k','k','k','k','k'], 
               levels = 12) #[0,10, 20,50,100,200,300,400])
ax.clabel(CS, inline=1, fontsize=10, fmt = '%i W')

ax.set_xlabel('$a_1, m/s^2$')
ax.set_ylabel('$a_2, m/s^2$')
ax.set_xlim(xlims)
ax.set_ylim(ylims)
orig_x = zero_a_simp_f(speed_simp)
rect = cuadrado(orig_x,0, 2* max_a_simp_f(0))
ax.add_patch(rect)
plt.plot(orig_x,0, 'ok')
plt.grid()



speed = max_speed_ax/2
max_a_val = max_a_f(speed)
min_a_val = min_a_f(speed)
max_a_lat_val = max_a_f(0)
acc_array_x = np.linspace(min_a_val, max_a_val, NN)
acc_array_y = np.linspace(-max_a_lat_val, max_a_lat_val, NN)
acc_XX, acc_YY = np.meshgrid(acc_array_x, acc_array_y)
total_pow_e_mat = total_pow_e_f(speed,0,acc_XX,acc_YY)
total_pow_e_mat_mod = total_pow_e_f_mod(speed,0,acc_XX,acc_YY)

ax = plt.subplot(1,3,2)
ax.set_title('Total electric power, with friction', picker=True)
#ax.contourf(phi_dot_mat_simp, tau_m_mat_simp, pow_e_mat_simp, levels = 30)
#plt.plot(speed_mat_simp[0,:],acc_mat_simp[0,:], 'k')
#plt.plot(speed_mat_simp[-1,:],acc_mat_simp[-1,:], 'k')
CS = ax.contour(acc_XX,acc_YY, total_pow_e_mat, 
#               colors = ['w','w','w','w','w','k','k','k','k','k'], 
               levels = 12) #[0,10, 20,50,100,200,300,400])
ax.clabel(CS, inline=1, fontsize=10, fmt = '%i W')

ax.set_xlabel('$a_1, m/s^2$')
ax.set_ylabel('$a_2, m/s^2$')
ax.set_xlim(xlims)
ax.set_ylim(ylims)
orig_x = zero_a_f(speed)
rect = cuadrado(orig_x,0, 2* max_a_f(0))
ax.add_patch(rect)
plt.plot(orig_x,0, 'ok')
plt.grid()

ax = plt.subplot(1,3,3)
ax.set_title('Total electric power, modified model', picker=True)
#ax.contourf(phi_dot_mat_simp, tau_m_mat_simp, pow_e_mat_simp, levels = 30)
#plt.plot(speed_mat_simp[0,:],acc_mat_simp[0,:], 'k')
#plt.plot(speed_mat_simp[-1,:],acc_mat_simp[-1,:], 'k')
CS = ax.contour(acc_XX,acc_YY, total_pow_e_mat_mod, 
#               colors = ['w','w','w','w','w','k','k','k','k','k'], 
               levels = 12) #[0,10, 20,50,100,200,300,400])
ax.clabel(CS, inline=1, fontsize=10, fmt = '%i W')

ax.set_xlabel('$a_1, m/s^2$')
ax.set_ylabel('$a_2, m/s^2$')
ax.set_xlim(xlims)
ax.set_ylim(ylims)
orig_x = zero_a_f(speed)
rect = cuadrado(orig_x,0, 2* max_a_f(0))
ax.add_patch(rect)
plt.plot(orig_x,0, 'ok')
plt.grid()

### Moving at half speed in $w_1$ and $w_2$

In [ ]:
NN = 100
speed_simp = max_speed_ax_simp/2
max_a_val_simp = max_a_simp_f(speed_simp)
min_a_val_simp = min_a_simp_f(speed_simp)
#max_a_lat_val_simp = max_a_simp_f(0)
acc_array_x_simp = np.linspace(min_a_val_simp, max_a_val_simp, NN)
acc_array_y_simp = np.linspace(min_a_val_simp, max_a_val_simp, NN)
acc_XX_simp, acc_YY_simp = np.meshgrid(acc_array_x_simp, acc_array_y_simp)
total_pow_e_mat_simp = total_pow_e_f_simp(speed_simp,speed_simp,acc_XX_simp,acc_YY_simp)

plt.figure(figsize = [22,7])
xlims = [-78, 28]
ylims = [-78, 28]

ax = plt.subplot(1,3,1)
ax.set_title('Total electric power, without friction', picker=True)
#ax.contourf(phi_dot_mat_simp, tau_m_mat_simp, pow_e_mat_simp, levels = 30)
#plt.plot(speed_mat_simp[0,:],acc_mat_simp[0,:], 'k')
#plt.plot(speed_mat_simp[-1,:],acc_mat_simp[-1,:], 'k')
CS = ax.contour(acc_XX_simp,acc_YY_simp, total_pow_e_mat_simp, 
#               colors = ['w','w','w','w','w','k','k','k','k','k'], 
               levels = 12) #[0,10, 20,50,100,200,300,400])
ax.clabel(CS, inline=1, fontsize=10, fmt = '%i W')

ax.set_xlabel('$a_1, m/s^2$')
ax.set_ylabel('$a_2, m/s^2$')
ax.set_xlim(xlims)
ax.set_ylim(ylims)
orig_x = zero_a_simp_f(speed_simp)
rect = cuadrado(orig_x,orig_x, 2* max_a_simp_f(0))
ax.add_patch(rect)
plt.plot(orig_x,orig_x, 'ok')
plt.grid()



speed = max_speed_ax/2
max_a_val = max_a_f(speed)
min_a_val = min_a_f(speed)
max_a_lat_val = max_a_f(0)
acc_array_x = np.linspace(min_a_val, max_a_val, NN)
acc_array_y = np.linspace(min_a_val, max_a_val, NN)
acc_XX, acc_YY = np.meshgrid(acc_array_x, acc_array_y)
total_pow_e_mat = total_pow_e_f(speed,speed,acc_XX,acc_YY)
total_pow_e_mat_mod = total_pow_e_f_mod(speed,speed,acc_XX,acc_YY)

ax = plt.subplot(1,3,2)
ax.set_title('Total electric power, with friction', picker=True)
#ax.contourf(phi_dot_mat_simp, tau_m_mat_simp, pow_e_mat_simp, levels = 30)
#plt.plot(speed_mat_simp[0,:],acc_mat_simp[0,:], 'k')
#plt.plot(speed_mat_simp[-1,:],acc_mat_simp[-1,:], 'k')
CS = ax.contour(acc_XX,acc_YY, total_pow_e_mat, 
#               colors = ['w','w','w','w','w','k','k','k','k','k'], 
               levels = 12) #[0,10, 20,50,100,200,300,400])
ax.clabel(CS, inline=1, fontsize=10, fmt = '%i W')

ax.set_xlabel('$a_1, m/s^2$')
ax.set_ylabel('$a_2, m/s^2$')
ax.set_xlim(xlims)
ax.set_ylim(ylims)
orig_x = zero_a_f(speed)
rect = cuadrado(orig_x,orig_x, 2* max_a_f(0))
ax.add_patch(rect)
plt.plot(orig_x,orig_x, 'ok')
plt.grid()

ax = plt.subplot(1,3,3)
ax.set_title('Total electric power, modified model', picker=True)
#ax.contourf(phi_dot_mat_simp, tau_m_mat_simp, pow_e_mat_simp, levels = 30)
#plt.plot(speed_mat_simp[0,:],acc_mat_simp[0,:], 'k')
#plt.plot(speed_mat_simp[-1,:],acc_mat_simp[-1,:], 'k')
CS = ax.contour(acc_XX,acc_YY, total_pow_e_mat_mod, 
#               colors = ['w','w','w','w','w','k','k','k','k','k'], 
               levels = 12) #[0,10, 20,50,100,200,300,400])
ax.clabel(CS, inline=1, fontsize=10, fmt = '%i W')

ax.set_xlabel('$a_1, m/s^2$')
ax.set_ylabel('$a_2, m/s^2$')
ax.set_xlim(xlims)
ax.set_ylim(ylims)
orig_x = zero_a_f(speed)
rect = cuadrado(orig_x,orig_x, 2* max_a_f(0))
ax.add_patch(rect)
plt.plot(orig_x,orig_x, 'ok')
plt.grid()

# Incluyendo giro

## Situación y ecuación principal

$$ Ha+Kw=R^T\Gamma$$
$$ \Gamma = R^{-T}(Ha+Kw)$$

In [ ]:
K, H, q_d, q_d_d

In [ ]:
gamma_rot = simplify(R_inv.T @ (H @ q_d_d + K @ q_d))
gamma_rot

In [ ]:
gamma_rot_num = gamma_rot.subs(sqrt(2),2**0.5).subs(r, 0.0667).subs(m,15.75).subs(I_w, 0.00266).subs(L_2, 0.29642).subs(m,15.75).subs(I_z, 0.461)
roundize(gamma_rot_num,3)

In [ ]:
q_d_w

In [ ]:
q_d_w_num = q_d_w.subs(sqrt(2),2**0.5).subs(r, 0.0667).subs(L_2, 0.29642)
roundize(q_d_w_num,2)

In [ ]:
roundize(pow_e_simp_num_nofric,3)

In [ ]:
roundize(pow_e_simp_num,3)

In [ ]:
total_p = 0
for iii in range(4):
    total_p += pow_e_simp_num_nofric.subs(phi_dot, q_d_w_num[iii]).subs(tau_m, gamma_rot_num[iii])
total_p = simplify(total_p)
roundize(total_p, 2)

In [ ]:
total_p_confric = 0
for iii in range(4):
    total_p_confric += pow_e_simp_num.subs(phi_dot, q_d_w_num[iii]).subs(tau_m, gamma_rot_num[iii])
total_p_confric = simplify(total_p_confric)
roundize(total_p_confric, 2)

In [ ]:
def total_pow_e_f_mod(w1, w2, a1, a2):
    p = 0
    for iii in range(4):
        p_motor = pow_e_simp_num.subs(phi_dot, q_d_w[iii]).subs(psi.diff(t,1),0).subs(tau_m, gamma[iii])
        p_motor = simplify(p_motor.subs(sqrt(2),2**0.5).subs(r, 0.0667).subs(L_2, 0.29642).subs(m,15.75).subs(I_w, 0.00266))
        p_motor_f = only_positive(lambdify([w_1, w_2, a_1, a_2], p_motor))
        p += p_motor_f(w1, w2, a1, a2)
    return p

### Maximas aceleraciones: t max y min f(phidot)

In [ ]:
gamma_impl = Matrix([symbols('tau_'+str(ii+1)) for ii in range(4)])
gamma_impl

In [ ]:
a_f_gamma = simplify(H_inv@(R.T @ gamma_impl - K@q_d))
a_f_gamma

Simple model

In [ ]:
max_gamma_simp = Matrix([0,0,0,0])
min_gamma_simp = Matrix([0,0,0,0])
zero_gamma_simp = Matrix([0,0,0,0])
for iii in range(4):
    max_gamma_simp[iii] = tau_m_v_max_num_simp.subs(phi_dot, q_d_w_num[iii])
    min_gamma_simp[iii] = tau_m_v_min_num_simp.subs(phi_dot, q_d_w_num[iii])
    zero_gamma_simp[iii] = tau_m_v_zero_num_simp.subs(phi_dot, q_d_w_num[iii])
roundize(max_gamma_simp,3),roundize(min_gamma_simp,3),roundize(zero_gamma_simp,3)

In [ ]:
max_a_simp = H_inv@(R.T@max_gamma_simp - K@q_d)
min_a_simp = H_inv@(R.T@min_gamma_simp - K@q_d)
zero_a_simp = H_inv@(R.T@zero_gamma_simp - K@q_d)
max_a_simp = max_a_simp.subs(sqrt(2),2**0.5).subs(r, 0.0667).subs(m,15.75).subs(I_w, 0.00266).subs(L_2, 0.29642).subs(I_z, 0.461)
min_a_simp = min_a_simp.subs(sqrt(2),2**0.5).subs(r, 0.0667).subs(m,15.75).subs(I_w, 0.00266).subs(L_2, 0.29642).subs(I_z, 0.461)
zero_a_simp = zero_a_simp.subs(sqrt(2),2**0.5).subs(r, 0.0667).subs(m,15.75).subs(I_w, 0.00266).subs(L_2, 0.29642).subs(I_z, 0.461)
roundize(max_a_simp,3),roundize(min_a_simp,3),roundize(zero_a_simp,3)

In [ ]:
max_a_simp_f_rot = lambdify([w_1, w_2, psi.diff(t)],max_a_simp[0])
min_a_simp_f_rot = lambdify([w_1, w_2, psi.diff(t)],min_a_simp[0])
zero_a_simp_f_rot = lambdify([w_1, w_2, psi.diff(t)],zero_a_simp[0])

In [ ]:
max_a_simp[0]-min_a_simp[0],max_a_simp[1]-min_a_simp[1]

In [ ]:
max_gamma = Matrix([0,0,0,0])
min_gamma = Matrix([0,0,0,0])
zero_gamma = Matrix([0,0,0,0])
for iii in range(4):
    max_gamma[iii] = tau_m_v_max_num.subs(phi_dot, q_d_w_num[iii])
    min_gamma[iii] = tau_m_v_min_num.subs(phi_dot, q_d_w_num[iii])
    zero_gamma[iii] = tau_m_v_zero_num.subs(phi_dot, q_d_w_num[iii])
roundize(max_gamma,3),roundize(min_gamma,3),roundize(zero_gamma,3)

In [ ]:
max_a = H_inv@(R.T@max_gamma - K@q_d)
min_a = H_inv@(R.T@min_gamma - K@q_d)
zero_a = H_inv@(R.T@zero_gamma - K@q_d)
max_a = simplify(max_a.subs(sqrt(2),2**0.5).subs(r, 0.0667).subs(m,15.75).subs(I_w, 0.00266).subs(L_2, 0.29642).subs(I_z, 0.461))
min_a = simplify(min_a.subs(sqrt(2),2**0.5).subs(r, 0.0667).subs(m,15.75).subs(I_w, 0.00266).subs(L_2, 0.29642).subs(I_z, 0.461))
zero_a = simplify(zero_a.subs(sqrt(2),2**0.5).subs(r, 0.0667).subs(m,15.75).subs(I_w, 0.00266).subs(L_2, 0.29642).subs(I_z, 0.461))
roundize(max_a,3),roundize(min_a,3),roundize(zero_a,3)

In [ ]:
max_a_f_rot = lambdify([w_1, w_2, psi.diff(t)],max_a[0])
min_a_f_rot = lambdify([w_1, w_2, psi.diff(t)],min_a[0])
zero_a_f_rot = lambdify([w_1, w_2, psi.diff(t)],zero_a[0])

In [ ]:
max_a[0]-min_a[0],max_a[1]-min_a[1]

## Consumo de energía para alcanzar una velocidad

In [ ]:
total_p_confric

In [ ]:
total_p

### Si no hay giro, a y w son las derivadas de x2

In [ ]:
x_2, y_2 = dynamicsymbols('x_2, y_2')


In [ ]:
x_2_d = x_2.diff()
x_2_dd = x_2_d.diff()
y_2_d = y_2.diff()
y_2_dd = y_2_d.diff()
x_2_dd, y_2_dd

In [ ]:
total_p_simp_irr = total_p.subs({a_1:x_2_dd, a_2:y_2_dd, w_1:x_2_d, w_2:y_2_d})
total_p_simp_irr

In [ ]:
integrate(total_p_simp_irr, t)

### Supongamos aceleración constante

In [ ]:
total_p_simp_irr = total_p.subs({w_1:a_1*t, w_2:a_2*t})
total_p_simp_irr

In [ ]:
__p = simplify(integrate(total_p_simp_irr, t)).subs(a_1**2 + a_2**2, a**2)
__p

In [ ]:
v_f = symbols('v_f')
roundize(expand(__p.subs(t, v_f/a)),3)

Modelo completo

In [ ]:
total_p_irr = total_p_confric.subs({w_1:a_1*t, w_2:a_2*t})
total_p_irr

In [ ]:
integrate(total_p_simp_irr, t)

si a_1>0 y a_2 >0

In [ ]:
simplify(total_p_irr.subs({sign(a_1*t): 1,sign(a_2*t): 1}))

In [ ]:
__p2 =simplify(integrate(simplify(total_p_irr.subs({sign(a_1*t): 1,sign(a_2*t): 1})),t)).subs(a_1**2, a**2 - a_2**2)
__p2

In [ ]:
v_f = symbols('v_f')
__p3 = collect(expand(__p2.subs(t, v_f/a)), v_f)
roundize(__p3, 3)

In [ ]:
g = symbols('\gamma')
g

In [ ]:
roundize(__p3.subs({a_1:a*cos(g), a_2:a*sin(g)}), 3)

In [ ]:
phi_dot_max = 2 * np.pi * 7000/(49*60)
phi_dot_max

In [ ]:
total_p_fun = lambdify([x_2.diff(), y_2.diff()], total_p)

In [ ]:
percent = [0, 0.25, 0.5, 0.75]


#plt.figure(figsize = [10,8])

psi_val = percent[0]
mx_spd = max_speed_axes_2(psi_val)
speed_arr = np.linspace(-mx_spd, mx_spd, 200)
xx_speed, yy_speed = np.meshgrid(speed_arr, speed_arr)
xy_power = total_p_fun(xx_speed, yy_speed)

fig, ax = plt.subplots(figsize = [8,8])
ax.set_title('Potencia eléctrica total, $\dot{\psi} = 0$', picker=True)
CS = ax.contour(speed_arr, speed_arr, xy_power)
ax.clabel(CS, inline=1, fontsize=10, fmt = '%i W')
ax.set_xlim([-0.8, 0.8])
ax.set_ylim([-0.8, 0.8])
ax.set_xlabel('$\dot{x}_2$, m/s')
ax.set_ylabel('$\dot{y}_2$, m/s')
rect = Rectangle((-mx_spd,-mx_spd), 2*mx_spd, 2*mx_spd, fill=False, linestyle = ':')
#pc = PatchCollection(errorboxes, facecolor=facecolor, alpha=alpha,edgecolor=edgecolor)

ax.add_patch(rect)
#plt.contour(speed_arr, speed_arr, xy_power)

#for per in percent:
#    plt.plot(gamma_arr,total_p_fun(gamma_arr, v_r_num* per), label = '$v_r$ = '+ str(per) + '$v_{r,max}$')
#plt.vlines(np.pi*np.array([0.25, 0.5, 0.75]), 0, 36, 'k', 'dotted')
plt.grid()
#plt.legend()
#plt.xlabel('$\gamma$, in radians')
#plt.ylabel('Total power, in W')

# Projection on axes 2

In [ ]:
H

In [ ]:
simplify(R_psi.T @ H @ R_psi)

In [ ]:
K

In [ ]:
simplify(R_psi.T @ K @ R_psi)

In [ ]:
q_d_2 = Matrix([x_2, y_2, psi]).diff()
q_d_d_2 = q_d_2.diff()
q_d_d_2, q_d_2

Case without acceleration: $\ddot{q}_2 = 0$

In [ ]:
K @ q_d_2

In [ ]:
Gamma_unif_2 = simplify(R_inv.T @ K @ q_d_2)
Gamma_unif_2

In [ ]:
Gamma_unif_2*r/sqrt(2)/I_w

In [ ]:
i = (tau_m + a * phi_dot + b * sign(phi_dot))/(k_e * mu * n)
i

In [ ]:
v = k_m * n * phi_dot + r_e * i
v

In [ ]:
pow_e = expand(i*v)
pow_e

In [ ]:
#pow_e = pow_e.replace(sign(phi_dot)**2, 1)
#pow_e_simp = collect(pow_e.replace(sign(phi_dot), 1), phi_dot)
pow_e_simp = collect(pow_e, phi_dot)
pow_e_simp

In [ ]:
pow_e_simp_num = pow_e_simp.subs(a, 0.01).subs(b, 0.4).subs(k_e, 0.0337).subs(k_m, 0.03148).subs(r_e, 1.84615).subs(n, 49).subs(mu, 0.6)
roundize(pow_e_simp_num,3)

In [ ]:
q_d_w_2 = simplify(R @ q_d_2)
q_d_w_2

In [ ]:
I_w

In [ ]:
total_p = 0
for iii in range(4):
    total_p += pow_e_simp_num.subs(phi_dot, q_d_w_2[iii]).subs(tau_m, Gamma_unif_2[iii]).subs(I_w, 0.00266)
total_p = simplify(total_p.subs(sqrt(2),2**0.5).subs(r, 0.0667).subs(L_2, 0.29642))
roundize(total_p, 2)

In [ ]:
total_p_fun = lambdify([x_2.diff(), y_2.diff(), psi_dot], total_p)

In [ ]:
percent = [0.1, 0.25, 0.5, 0.75]


plt.figure(figsize = [15,16])

for iii in range(4):
    psi_val = percent[iii]*psi_dot_max
    mx_spd = max_speed_axes_2(psi_val)
    speed_arr = np.linspace(-mx_spd, mx_spd, 200)
    xx_speed, yy_speed = np.meshgrid(speed_arr, speed_arr)
    xy_power = total_p_fun(xx_speed, yy_speed, psi_val)

    ax = plt.subplot(2,2,iii+1)
    title = 'Potencia eléctrica total, $\dot{\psi} = '+ str(percent[iii])+'\dot{\psi}_{max}$'
    ax.set_title(title, picker=True)
    CS = ax.contour(speed_arr, speed_arr, xy_power)
    ax.clabel(CS, inline=1, fontsize=10, fmt = '%1.2f W')
    ax.set_xlim([-0.8, 0.8])
    ax.set_ylim([-0.8, 0.8])
    rect = Rectangle((-mx_spd,-mx_spd), 2*mx_spd, 2*mx_spd, fill=False, linestyle = ':')
    #pc = PatchCollection(errorboxes, facecolor=facecolor, alpha=alpha,edgecolor=edgecolor)

    ax.add_patch(rect)
    #plt.contour(speed_arr, speed_arr, xy_power)

    #for per in percent:
    #    plt.plot(gamma_arr,total_p_fun(gamma_arr, v_r_num* per), label = '$v_r$ = '+ str(per) + '$v_{r,max}$')
    #plt.vlines(np.pi*np.array([0.25, 0.5, 0.75]), 0, 36, 'k', 'dotted')
    plt.grid()
#plt.legend()
#plt.xlabel('$\gamma$, in radians')
#plt.ylabel('Total power, in W')

In [ ]:
6%5

In [ ]:
percent = np.linspace(0, 1, 21)
viridis = cm.get_cmap('viridis', 12)
plt.figure(figsize = [10,8])
for iii in range(len(percent)):
    psi_val = percent[iii]*psi_dot_max
    mx_spd = max_speed_axes_2(psi_val)
    speed_arr = np.linspace(0, mx_spd, 200)
    x_power = total_p_fun(speed_arr, 0, psi_val)
    if iii%5 == 0:
        title = '$\dot{\psi} = '+ str(round(percent[iii],2))+'\dot{\psi}_{max}$'
        plt.plot(speed_arr, x_power, color = viridis(iii/len(percent)),label = title)
    else:
        plt.plot(speed_arr, x_power, color = viridis(iii/len(percent)))
plt.legend()
plt.title('Power when $\dot{y_2}=0$')
plt.xlabel('$\dot{x}_2$, m/s')
plt.ylabel('power, W')
plt.grid()

In [ ]:
percent = np.linspace(0, 1, 21)
viridis = cm.get_cmap('viridis', 12)
plt.figure(figsize = [10,8])
for iii in range(len(percent)):
    psi_val = percent[iii]*psi_dot_max
    mx_spd = max_speed_axes_2(psi_val)
    speed_arr = np.linspace(0, mx_spd, 200)
    x_power = total_p_fun(speed_arr, speed_arr, psi_val)
    if iii%5 == 0:
        title = '$\dot{\psi} = '+ str(round(percent[iii],2))+'\dot{\psi}_{max}$'
        plt.plot(speed_arr, x_power, color = viridis(iii/len(percent)),label = title)
    else:
        plt.plot(speed_arr, x_power, color = viridis(iii/len(percent)))
plt.legend()
plt.title('Power when $\dot{x_2}=\dot{y_2}$')
plt.xlabel('$\dot{x}_2$, m/s')
plt.ylabel('power, W')
plt.grid()

## Electric Motor Model

$$ V = K_m \dot{\phi} + Ri$$
$$ \tau_m = K_ei - \tau_r$$

In [ ]:
def friction(a, b, phi_dot, V, eps_phi = 0.01):
    fric_wet = a * phi_dot
    fric_dry = np.where(npor(
        phi_dot>eps_phi,
        npand(
            np.abs(phi_dot)<=eps_phi,
            V > 0)
    ),b, 0)
    fric_dry = np.where(npor(
        phi_dot< -eps_phi,
        npand(
            np.abs(phi_dot)<=eps_phi,
            V < 0)
    ),-b, fric_dry)
    return fric_wet + fric_dry

In [ ]:
def create_motor_model(i_7000 = 0.5):
    V = 24
    R = 24/13
    K_m = (V - R*i_7000)/(2 * np.pi*7000/60)
    t_rat = 4300*9.8/(100*1000)    #570*9.8/(100*1000)
    phi_rat = 0                    #5900*np.pi*2/60
    i_rat = (V - K_m*phi_rat)/R
    K_e = t_rat/(i_rat-i_7000)
    t_r = -t_rat + K_e * i_rat
    print(f'K_m = {K_m}, K_e = {K_e}, R = {R}, i_rat = {i_rat}, t_r = {t_r}')
    def motor_model(phi_dot, V = 24):
        i = (V - K_m*phi_dot)/R
        t = K_e * i - friction(0,t_r, phi_dot, V)
        p = t * phi_dot
        ef = p / (i * V)
        return t, i, p, ef
    def simp_motor_model(phi_dot, V = 24):
        i = (V - K_m*phi_dot)/R
        t = K_e * i - friction(0,t_r, phi_dot, V)
        return t
    return motor_model, simp_motor_model

In [ ]:
x = np.linspace(0, 2 * np.pi * 7000/60, 300)
n = x * 30 / np.pi
motor_model = create_motor_model(0.5)[0]
t, i, p, ef = motor_model(x)
t = t *100000/9.8

fig, ax1 = plt.subplots()

ax1.plot(t,n)
ax1.set_ylim([0,7000])
ax1.set_xlim([0,4400])

ax2 = ax1.twinx()
ax2.plot(t,i)
ax2.set_ylim([0,13])

ax3 = ax1.twinx()
ax3.plot(t,p)
ax3.spines["right"].set_position(("axes", 1.1))
ax3.set_ylim([0,80])

ax4 = ax1.twinx()
ax4.plot(t,ef)
ax4.spines["right"].set_position(("axes", 1.2))
ax4.set_ylim([0,1])
plt.grid()
plt.title('Motor Curves')

plt.grid(True)

### Comparison with manufacturer original motor curves:
![Original curves](images_dejabot/curvas_motor.png)

Expanded curves outside normal regime:

In [ ]:
x = np.linspace(-2 * np.pi * 7000/60, 4 * np.pi * 7000/60, 300)
n = x * 30 / np.pi
motor_model = create_motor_model(0.5)[0]
t, i, p, ef = motor_model(x)
t = t *100000/9.8

fig, ax1 = plt.subplots()

ax1.plot(t,n, 'r')
ax1.set_ylim([-14000,14000])
ax1.set_xlim([-4400,8800])

ax2 = ax1.twinx()
ax2.plot(t,i)
ax2.set_ylim([-26,26])

ax3 = ax1.twinx()
ax3.plot(t,p)
ax3.spines["right"].set_position(("axes", 1.1))
ax3.set_ylim([-80,80])

ax4 = ax1.twinx()
ax4.plot(t,ef, 'g')
ax4.spines["right"].set_position(("axes", 1.2))
ax4.set_ylim([-2,2])
plt.grid()

plt.grid(True)

Adapted Curves with reductor gears:

## Electric Motor Model

$$ V = K_m \dot{\phi_{mot}} + Ri$$
$$ \tau_m = N K_ei\mu_{trans} - \tau_r$$

$$ \dot{\phi}_{mot} = N \dot{\phi}$$
$$ \tau_r = a \dot{\phi} + b·sign(\dot{\phi}) $$

In [ ]:
def create_motoreduct_model(i_7000 = 0.5, N = 49, eff = 0.6, a = 0.01, b = 0.4):
    V = 24
    R = 24/13
    K_m = (V - R*i_7000)/(2 * np.pi*7000/60)
    t_rat = 4300*9.8/(100*1000)    #570*9.8/(100*1000)
    phi_rat = 0                    #5900*np.pi*2/60
    i_rat = (V - K_m*phi_rat)/R
    K_e = t_rat/(i_rat-i_7000)
    
    print(f'K_m = {K_m}, K_e = {K_e}, R = {R}, i_rat = {i_rat}')
    eps_phi = 0.1
    
    def motoreduct_model(phi_dot, V = 24):
        phi_dot_motor = phi_dot * N
        i = (V - K_m*phi_dot_motor)/R
        t = K_e * i * N * eff - friction(a, b, phi_dot, V, eps_phi)
        p = t * phi_dot
        ef = p / (i * V)
        return t, i, p, ef
    def simp_motoreduct_model(phi_dot, V = 24):
        phi_dot_motor = phi_dot * N
        i = (V - K_m*phi_dot_motor)/R
        t = K_e * i * N * eff - friction(a, b, phi_dot, V, eps_phi)
        return t
    return motoreduct_model, simp_motoreduct_model

In [ ]:
x = np.linspace(0, 2 * np.pi * 7000/(49*60), 300)
n = x * 30 / np.pi
motor_model = create_motoreduct_model(0.5)[0]
t, i, p, ef = motor_model(x)
t = t *100000/9.8

fig, ax1 = plt.subplots()

ax1.plot(t,n)
ax1.set_ylim([0,150])
ax1.set_xlim([0,130000])

ax2 = ax1.twinx()
ax2.plot(t,i)
ax2.set_ylim([0,13])

ax3 = ax1.twinx()
ax3.plot(t,p)
ax3.spines["right"].set_position(("axes", 1.1))
ax3.set_ylim([0,80])

ax4 = ax1.twinx()
ax4.plot(t,ef)
ax4.spines["right"].set_position(("axes", 1.2))
ax4.set_ylim([0,1])
#plt.grid()

ax5 = ax1.twinx()
ax5.spines["right"].set_position(("axes", -0.2))
ax5.set_ylim([0, x[-1]*0.0667])

plt.grid(True)

In [ ]:
def torq_lims(phi_dot, v_max = 24, motor = create_motoreduct_model()[1]):
    t_max = motor(phi_dot, v_max)
    t_min = motor(phi_dot, -v_max)
    
    return t_max, t_min

In [ ]:
def vect_torq_lims(geom, q, q_dot):
    a = get_a(geom, q)
    phi_dot = a.T@q_dot
    #print(f'phi_dot: {np.round(phi_dot,3)}')
    t_max = []
    t_min = []
    for wheel in phi_dot:
        maxt, mint = torq_lims(wheel)
        t_max.append(maxt)
        t_min.append(mint)
    t_max = np.array(t_max)
    t_min = np.array(t_min)
    
    return (t_max, t_min)

### Simplified expression linking maximum and mínimum torque as function of speed

In [ ]:
phi_arr = np.linspace(-20, 20, 500)
max_t, min_t = torq_lims(phi_arr)
t_v_med, t_min_v_med = torq_lims(phi_arr, v_max= 12)
t_v_0, _ = torq_lims(phi_arr, v_max= 0)
max_t_nodry, min_t_nodry = torq_lims(phi_arr, motor = create_motoreduct_model(b=0)[1])
t_v_med_nodry, t_min_v_med_nodry = torq_lims(phi_arr, v_max= 12, motor = create_motoreduct_model(b=0)[1])
t_v_0_nodry, _ = torq_lims(phi_arr, v_max= 0, motor = create_motoreduct_model(b=0)[1])

In [ ]:
plt.figure(figsize=[15,12])
n_arr = phi_arr * 30 / np.pi 
plt.plot(n_arr, max_t*100/9.8, label = 'par máximo, modelo completo')
plt.plot(n_arr, min_t*100/9.8, label = 'par mínimo, modelo completo')
plt.plot(n_arr, max_t_nodry*100/9.8, label = 'par máximo, sin fricción seca')
plt.plot(n_arr, min_t_nodry*100/9.8, label = 'par mínimo, sin fricción seca')
plt.plot(n_arr, t_v_med*100/9.8, label = 'par a 12 V, modelo completo')
plt.plot(n_arr, t_min_v_med*100/9.8, label = 'par a -12 V, modelo completo')
plt.plot(n_arr, t_v_med_nodry*100/9.8, label = 'par a 12 V, sin fricción seca')
plt.plot(n_arr, t_min_v_med_nodry*100/9.8, label = 'par a -12 V, sin fricción seca')
plt.plot(n_arr, t_v_0*100/9.8, label = 'par a 0 V, modelo completo')
plt.plot(n_arr, t_v_0_nodry*100/9.8, label = 'par a 0 V, sin fricción seca')
plt.hlines(0, -200, 200, 'k', 'dotted')
plt.vlines(0, -300, 300, 'k', 'dotted')
plt.grid
plt.xlabel('velocidad del motor, RPM')
plt.ylabel('par motor, Kg·cm')
plt.legend()

In [ ]:
def cutpoint(x1, x2, y1, y2):
    a = (y1-y2)/(x1-x2)
    b = y1 - a * x1
    return -b/a

In [ ]:
cut1 = cutpoint(phi_arr[260], phi_arr[499], max_t[260], max_t[499])
cut2 = cutpoint(phi_arr[260], phi_arr[499], max_t_nodry[260], max_t_nodry[499])
corr_factor = cut2/cut1
corr_factor

In [ ]:
cut1

In [ ]:
new_a = 0.4/cut1 + 0.01
new_a

In [ ]:
friction(0.01, 0.4, cut1, 24)

In [ ]:
friction(new_a , 0.0, cut1, 24)

new_a = (corr_factor -1)* 0.0314812 / 1.846154 + corr_factor* 0.01
new_a

In [ ]:
max_t_new_m, min_t_new_m = torq_lims(phi_arr, motor = create_motoreduct_model(b=0, a=new_a)[1])
t_v_med_new_m, t_min_v_med_new_m = torq_lims(phi_arr, v_max= 12, motor = create_motoreduct_model(b=0, a=new_a)[1])
t_v_0_new_m, _ = torq_lims(phi_arr, v_max= 0, motor = create_motoreduct_model(b=0, a=new_a)[1])

In [ ]:
plt.figure(figsize=[15,12])
n_arr = phi_arr * 30 / np.pi 
plt.plot(n_arr, max_t*100/9.8, label = 'par máximo, modelo completo')
plt.plot(n_arr, min_t*100/9.8, label = 'par mínimo, modelo completo')
plt.plot(n_arr, max_t_new_m*100/9.8, label = 'par máximo, modificado')
plt.plot(n_arr, min_t_new_m*100/9.8, label = 'par mínimo, modificado')
plt.plot(n_arr, t_v_med*100/9.8, label = 'par a 12 V, modelo completo')
plt.plot(n_arr, t_min_v_med*100/9.8, label = 'par a -12 V, modelo completo')
plt.plot(n_arr, t_v_med_new_m*100/9.8, label = 'par a 12 V, modificado')
plt.plot(n_arr, t_min_v_med_new_m*100/9.8, label = 'par a -12 V, modificado')
plt.plot(n_arr, t_v_0*100/9.8, label = 'par a 0 V, modelo completo')
plt.plot(n_arr, t_v_0_new_m*100/9.8, label = 'par a 0 V, modificado')
plt.hlines(0, -200, 200, 'k', 'dotted')
plt.vlines(0, -300, 300, 'k', 'dotted')
plt.grid
plt.xlabel('velocidad del motor, RPM')
plt.ylabel('par motor, Kg·cm')
plt.legend()

In [ ]:
max_t_no_f, min_t_no_f = torq_lims(phi_arr, motor = create_motoreduct_model(b=0, a=0)[1])
t_v_med_no_f, t_min_v_med_no_f = torq_lims(phi_arr, v_max= 12, motor = create_motoreduct_model(b=0, a=0)[1])
t_v_0_no_f, _ = torq_lims(phi_arr, v_max= 0, motor = create_motoreduct_model(b=0, a=0)[1])

In [ ]:
plt.figure(figsize=[15,12])
n_arr = phi_arr * 30 / np.pi 
plt.plot(n_arr, max_t*100/9.8, label = 'par máximo, modelo completo')
plt.plot(n_arr, min_t*100/9.8, label = 'par mínimo, modelo completo')
plt.plot(n_arr, max_t_no_f*100/9.8, label = 'par máximo, sin rozamiento')
plt.plot(n_arr, min_t_no_f*100/9.8, label = 'par mínimo, sin rozamiento')
plt.plot(n_arr, t_v_med*100/9.8, label = 'par a 12 V, modelo completo')
plt.plot(n_arr, t_min_v_med*100/9.8, label = 'par a -12 V, modelo completo')
plt.plot(n_arr, t_v_med_no_f*100/9.8, label = 'par a 12 V, sin rozamiento')
plt.plot(n_arr, t_min_v_med_no_f*100/9.8, label = 'par a -12 V, sin rozamiento')
plt.plot(n_arr, t_v_0*100/9.8, label = 'par a 0 V, modelo completo')
plt.plot(n_arr, t_v_0_no_f*100/9.8, label = 'par a 0 V, sin rozamiento')
plt.hlines(0, -200, 200, 'k', 'dotted')
plt.vlines(0, -300, 300, 'k', 'dotted')
plt.grid
plt.xlabel('velocidad del motor, RPM')
plt.ylabel('par motor, Kg·cm')
plt.legend()